In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import tensorflow as tf

import ND_north_star.src.noise_generators.perlin_noise_generator as PNG
import ND_north_star.src.edge_detection.complex as ED_com
import ND_north_star.src.edge_detection.gradient_detection as ED_sim


## Generate dataset

In [3]:
resolution = 30
size = 1000

In [4]:
dict = PNG.normalized_perlin_data([resolution, resolution], octaves=4)

In [5]:
data = [PNG.ND_perlin_matrix([resolution, resolution], octaves=2, noise_cutoff_list=[0.5, 0, 1]) for _ in range(size)]

In [6]:
def perlin_M_to_coords(perlin_matrix):
    # Initialize empty lists for coordinates and values
    coordinates = []
    values = []

    # Helper function to recursively flatten the nested data
    def recursive_flatten(current_data, current_coords):
        for i, item in enumerate(current_data):
            if isinstance(item, list):
                # If the item is a list, recursively flatten it
                recursive_flatten(item, current_coords + [i])
            else:
                # If the item is a value, store its coordinates and value
                coordinates.append(current_coords + [i])
                values.append(item)

    # Start the recursive flattening process
    recursive_flatten(perlin_matrix, [])

    # Determine the number of dimensions
    num_dimensions = max(map(len, coordinates))

    # Create separate arrays for each dimension
    coordinate_arrays = [np.array([coord[dim] if dim < len(coord) else 0 for coord in coordinates]) for dim in range(num_dimensions)]

    # Convert the values list to a NumPy array
    values_array = np.array(values)

    return coordinate_arrays, values_array


In [7]:
def drop_coords(input_coord_array, values, drop_perc=0.5):
    num_points = len(input_coord_array[0])
    num_points_to_drop = int(num_points * drop_perc)
    
    drop_indices = np.random.choice(num_points, num_points_to_drop, replace=False)

    keep_coord_array = [np.delete(coord, drop_indices, axis=0) for coord in input_coord_array]
    keep_values = np.delete(values, drop_indices, axis=0)

    return keep_coord_array, keep_values

In [8]:
X = np.zeros([size, resolution, resolution])
for i in range(size):
    coord_array, values = perlin_M_to_coords(data[i])
    dropped_coords_array_50, dropped_values_50 = drop_coords(coord_array, values, 0.5)
    dropped_img = np.zeros([resolution, resolution])
    for j in range(len(dropped_values_50)):
        x = dropped_coords_array_50[0][j]
        y = dropped_coords_array_50[1][j]
        dropped_img[x][y] = dropped_values_50[j]
        X[i] = dropped_img

In [9]:
y = [ED_sim.gradient_edge_detection(data[i]) for i in range(size)]

In [107]:
# X_sparse = [tf.sparse.from_dense(image) for image in X]
# y_sparse = [tf.sparse.from_dense(image) for image in y]

## Train test split

In [39]:
# X=data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2345)

## CNN

In [12]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [13]:
model = models.Sequential([
    layers.Conv2D( 32, (3,3), activation='relu', input_shape=(resolution,resolution,1,), padding = 'same'),
    layers.MaxPooling2D( (2,2), strides = 2 ),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D( (2,2), strides=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(resolution*resolution, activation='softmax'),
    layers.Reshape((resolution, resolution))
])

/Users/yangxiaoluo/miniconda3/envs/nd-path-finder/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [15]:
X_train_tensor = tf.convert_to_tensor(X_train)
y_train_tensor = tf.convert_to_tensor(y_train)
X_test_tensor = tf.convert_to_tensor(X_test)
y_test_tensor = tf.convert_to_tensor(y_test)

In [16]:
epochs=500
history = model.fit(X_train_tensor, y_train_tensor, validation_data=(X_test_tensor, y_test_tensor),
                      epochs=epochs, 
                      batch_size=512)

Epoch 1/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - loss: 0.1352 - mse: 0.1352 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 2/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.1351 - mse: 0.1351 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 3/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1351 - mse: 0.1351 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 4/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1352 - mse: 0.1352 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 5/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1351 - mse: 0.1351 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 6/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.1352 - mse: 0.1352 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 7/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.1352 - mse: 0.1352 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 8/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.1350 - mse: 0.1350 - val_loss: 0.1365 - val_mse: 0.1365
Epoch 9/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.1354 - mse: